In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

alphabetical_list = 'J'

def get_legislation_links(soup):
    legislation_links = []
    
    # Find the <ul> container containing the legislation links
    ul_container = soup.find('ul')
    
    # Get all the <li> tags within the <ul> container
    li_tags = ul_container.find_all('li')
    
    # Iterate through the <li> tags and extract the links
    for li in li_tags:
        a_tag = li.find('a')
        if a_tag:
            legislation_links.append(a_tag['href'])
    
    return legislation_links

def get_section_links(soup):
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and href.startswith('s'):
            links.append(href)
    return links

# Define the function to extract title and subsection number
def extract_title_and_subsection(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    pre_container = soup.find('pre')

    subsection_elements = pre_container.find_all(lambda tag: tag.name == 'a' and tag.has_attr('href') and not tag.has_attr('name'))
    subsection_numbers = [a.text.strip() for a in subsection_elements]
    
    subsection_names = []
    for a in subsection_elements:
        next_sibling = a.next_sibling
        if isinstance(next_sibling, str):
            subsection_names.append(next_sibling.strip())

    act_title = soup.find('h3').text.strip()

    return subsection_numbers, subsection_names, act_title



data = {'Act': [],'Section_number':[], 'Section_title': [] , 'Section_text': []}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

for letter in alphabetical_list:
    toc_url = f'http://classic.austlii.edu.au/au/legis/act/consol_act/toc-{letter}.html'
    response = requests.get(toc_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    legislation_links = get_legislation_links(soup)
    print(f"Letter {letter} - Legislation links: {legislation_links}")

    for leg_link in legislation_links:
        leg_url = 'http://classic.austlii.edu.au/au/legis/act/consol_act/' + leg_link + '/'
        print(leg_link)
        response = requests.get(leg_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        act_title = soup.find('title').text

        section_links = get_section_links(soup)
        print(f"Act {act_title} - Section links: {section_links}")
        
      
        
        
        for sec_link in section_links:
            sec_url = base_url + leg_link + '/' + sec_link
            response = requests.get(sec_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            cleaned_text = []
            act = soup.find('h3').text.strip().split(' - ')[0]
            acts = soup.find('h3').text
            section_number = acts.split()[-1]
            section_title = soup.find('b').text
            print(section_title)
            section_text = soup.find_all('p')
            for section in section_text:
                text = section.get_text().strip()
                text = ' '.join(text.split())  # remove extra whitespaces
                text = re.sub('<.*?>', '', text)  # remove HTML tags
                cleaned_text.append(text)
            print(cleaned_text)
            data['Section_text'].append(cleaned_text)
            data['Act'].append(act)
            data['Section_title'].append(section_title)
            data['Section_number'].append(section_number)
            print(data)
df = pd.DataFrame(data)
print(df)